In [1]:
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics 

/home/keisuke/Documents/Research/sshfs/DimerMaster/scripts/functions.py:65: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [2]:
from conf import *
from numba import njit, jit

In [3]:
h = 1.0
V = 1.0

length = [8, 8]
name = 'h={}V={}l={}'.format(h, V, length)
kernel = 1
n_chains = 1
n_discards = 1
n_samples = 100

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))


ma = nk.machine.RbmDimer(hi, hex_, alpha = alpha, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)

ma.load('save/ma/'+name)


sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition, length = length, n_chains=n_chains, sweep_size = 70, kernel = 1)
sa.generate_samples(n_discards) # discard the begginings of metropolis sampling.
print('discard samples')
samples_state = sa.generate_samples(int(n_samples/n_chains))
samples_state = samples_state.reshape(-1, ma.hilbert.size)

print('prepared initial samples')

4
3.970005750656128 for metropolis
discard samples
0.16279172897338867 for metropolis
prepared initial samples


In [4]:
from scripts import dynamics as dy
from importlib import reload
reload(dy)

d = dy.new_dynamics(op, ma)
import time

In [5]:
t_list = np.linspace(0, 30, 11)

In [6]:
samples_state.shape

(100, 128)

In [24]:
%time P, T = d.dynamics(samples_state, 10)

0.4884757995605469 cal log
0.5672433376312256 cal log
0.6599569320678711 cal log
0.7263798713684082 cal log
0.7977445125579834 cal log
0.856661319732666 cal log
0.9028880596160889 cal log
0.9672219753265381 cal log
1.0219569206237793 cal log
1.0752029418945312 cal log
CPU times: user 8.08 s, sys: 12 ms, total: 8.1 s
Wall time: 8.07 s


In [7]:
T_ = T.T
uni, ind = np.unique(T_, return_index=True)

In [57]:
X = samples_state.astype(np.int8)

In [74]:
log_val_kernel = ma._log_val_kernel
_w = ma._w
_a = ma._a
_b = ma._b
_r = ma._r

In [85]:
sections = np.ones(X.shape[0])

In [77]:
s = time.time()
log_val_prime = np.real(log_val_kernel(X.astype(np.float64), None, _w, _a, _b, _r))
print(time.time()-s)

0.025664091110229492


In [333]:
@jit(fastmath=True)
def numpy_log_cosh_sum(x, out):
#     x = x * np.sign(x.real)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            out[i] += x[i,j] - np.log(2.0) + np.log(1.0 + np.exp(-2.0 * x[i,j]))
    return out

@jit(fastmath=True)
def _log_cosh_sum(x, out):
    x = x * np.sign(x.real)
    for i in range(x.shape[0]):
        out[i] = np.sum(x[i] - np.log(2.0) + np.log(1.0 + np.exp(-2.0 * x[i])))
    return out

@njit
def _dot(X, w, out):
    
    for i in range(X.shape[0]):
        for j in range(w.shape[1]):
#             out_ij = out[i,j]
            for l in range(X.shape[1]):
                out[i,j] += X[i,l] * w[l,j]
    return out
    
    
    

In [345]:
%%time 
    out = np.empty(X.shape[0], dtype=np.float64)
    _log_cosh_sum(r, out)

CPU times: user 19 ms, sys: 0 ns, total: 19 ms
Wall time: 18.9 ms


array([0.7654612 , 0.76811602, 0.76309587, ..., 0.75646857, 0.77094005,
       0.75896698])

In [349]:
%%time 
    out = np.empty(X.shape[0], dtype=np.float64)
    numpy_log_cosh_sum(r, out)

CPU times: user 15.4 ms, sys: 0 ns, total: 15.4 ms
Wall time: 15.4 ms


array([0.7654612 , 0.76811602, 0.76309587, ..., 0.76520739, 0.77656835,
       0.75908644])

In [327]:
out = np.empty(X.shape[0], dtype=np.float64)

In [328]:
X.dot(_w)

array([[ 0.00535431, -0.08177295,  0.06112185, ..., -0.06396568,
         0.02583953,  0.02139704],
       [-0.00981531,  0.26920866,  0.281769  , ...,  0.1109706 ,
         0.1070953 ,  0.03293422],
       [ 0.01600842,  0.11044312,  0.01940674, ...,  0.1530982 ,
         0.11114629, -0.00498917],
       ...,
       [ 0.18495536,  0.00642587, -0.1271401 , ...,  0.10302041,
        -0.09885506, -0.09689404],
       [ 0.1054339 , -0.02622662,  0.01161515, ...,  0.04229888,
        -0.0183555 , -0.19516845],
       [ 0.06722916, -0.11148691,  0.0695765 , ...,  0.05926635,
        -0.01534759,  0.05655404]])

In [329]:
out_w = np.zeros((X.shape[0], _w.shape[1]), dtype=np.float64)
_dot(X, _w, out_w)

array([[ 0.00535431, -0.08177295,  0.06112185, ..., -0.06396568,
         0.02583953,  0.02139704],
       [-0.00981531,  0.26920866,  0.281769  , ...,  0.1109706 ,
         0.1070953 ,  0.03293422],
       [ 0.01600842,  0.11044312,  0.01940674, ...,  0.1530982 ,
         0.11114629, -0.00498917],
       ...,
       [ 0.18495536,  0.00642587, -0.1271401 , ...,  0.10302041,
        -0.09885506, -0.09689404],
       [ 0.1054339 , -0.02622662,  0.01161515, ...,  0.04229888,
        -0.0183555 , -0.19516845],
       [ 0.06722916, -0.11148691,  0.0695765 , ...,  0.05926635,
        -0.01534759,  0.05655404]])

In [273]:
# %%time    

r = X.dot(_w)
numpy_log_cosh_sum(r, out)

array([6.94676642e-310, 4.65639296e-310, 4.65639060e-310, ...,
       0.00000000e+000, 4.65639084e-310, 0.00000000e+000])

In [106]:
out

array([0.7654612 , 0.76811602, 0.76309587, ..., 0.75646857, 0.77094005,
       0.75896698])

In [111]:
%%time
ma.log_val(X)

CPU times: user 285 ms, sys: 20 ms, total: 305 ms
Wall time: 26 ms


array([0.7654612 +0.j, 0.76811602+0.j, 0.76309587+0.j, ...,
       0.75646857+0.j, 0.77094005+0.j, 0.75896698+0.j])

In [10]:
t_list = np.linspace(0, 10, 11)

In [11]:
T_ind = np.argwhere(np.logical_and(T[:,:,None] <t_list[None,None,1:], t_list[None,None,:-1]<=T[:,:,None]))

In [12]:
T_ind[:,[0,1]][:,0], T_ind[:,[0,1]][:,1]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
        2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
        4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
        6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
        8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [13]:
T_ind[:,2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
P[T_ind[:,[0,1]][:,0], T_ind[:,[0,1]][:,1]]

array([[ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1, -1],
       [ 1, -1,  1, ...,  1,  1, -1],
       ...,
       [ 1, -1,  1, ..., -1, -1, -1],
       [-1, -1,  1, ...,  1, -1, -1],
       [-1,  1,  1, ...,  1, -1, -1]], dtype=int8)

In [15]:
ind

array([ 0, 91, 31, 21, 41, 11, 92, 32, 81, 12, 71, 22, 82, 51, 42,  1, 43,
       72, 52, 53, 83, 13, 23, 14, 73, 33, 44, 93, 74, 75, 54, 15, 24, 94,
       34, 61, 84, 85, 55, 76,  2, 45, 35, 25, 62, 77, 63, 64, 78, 95, 26,
       96, 86, 97, 46, 16, 65, 56, 57, 66, 27, 28, 79, 98, 67, 29, 58,  3,
       36, 37, 47, 59, 48, 99,  4,  5, 17, 38, 39, 68, 49, 18, 19, 69, 87,
       88, 89,  6,  7,  8,  9])

In [16]:
num = 2500
batch_size = samples_state.shape[0]
P = []
for i in range(int(1)):
    s = time.time()
    print(i * num ,(i+1)*num)
    P.append(d.dynamics(samples_state[i * num : (i+1)*num], t_list, 0))
    print('{}th loop done : {}'.format(i, time.time()-s))
P = np.vstack(P)

0 2500


TypeError: dynamics() takes 3 positional arguments but 4 were given

In [ ]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


l = []
for i in range(length[0]):
    for j in range(length[1]):
        l.append([i, j])
l = np.array(l)

a_ = [a for _ in range(np.prod(length))]
a = np.array(a_)

edges, colors = hex_.dimer_corr(l,a)
operators = f.return_dimer_operator(hi, edges, colors)
sections = np.arange(P.shape[0])

_, mels1 = operators[0].get_conn_flattened(P[:,0,:], sections)
sub1 = operators[0].get_conn_flattened(P[:,0,:], sections)[1].mean().real

dimer_corr = np.zeros((length[0],length[1],t_list.shape[0]))
dimer_std = np.zeros((length[0],length[1],t_list.shape[0]))

for l1 in range(length[0]):
    for l2 in range(length[1]):
        sub2 = operators[1].get_conn_flattened(P[:,0,:], sections)[1].mean().real 
        for i in range(t_list.shape[0]):
            _, mels2 = operators[l1 * length[1] + l2].get_conn_flattened(P[:,i,:], sections)
            dimer_corr[l1,l2,i] = np.real(((mels1 * mels2 ).mean()))
            dimer_std[l1,l2,i] = np.real(((mels1 * mels2 ).std()))
        dimer_corr[l1,l2] -= sub1 * sub2
        dimer_std[l1,l2] /= np.sqrt(P.shape[0])

In [ ]:
dimer_corr2 = dimer_corr

In [ ]:
dimer_corr2[1,3]

In [ ]:
dimer_corr[1,3]

In [ ]:
dimer_std[1,3]

In [ ]:
t_list